In [77]:
import os
from dotenv import load_dotenv
from llama_index.llms.llama_api import LlamaAPI

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')

api_key = os.getenv('LLAMA_API_KEY')
llm = LlamaAPI(api_key=api_key)


In [66]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import StorageContext

username = "neo4j"
password = os.getenv('NEO4J_PW')
url = os.getenv('NEO4J_URL')
database = "neo4j"

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [53]:
from llama_index.core.indices.knowledge_graph.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    with_nl2graphquery=True,
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery,
)

In [92]:
query = "tell me in detail about sirius black"

response = query_engine_with_nl2graphquery.query(
    query,
)

if response.metadata is None:
    # 1) answer needs to be queried from Llama2 endpoint
    external_response = str(llm.complete(query))
    print("From Llama2: " + external_response)

    # 2) answer needs to be stored into txt file for loading into KG
    with open('data/external_response.txt', 'w') as f:
        print(external_response, file=f)
else:
    print("From Graph: " + str(response))
    print(list(list(response.metadata.values())[0]["kg_rel_map"].keys()))

From Llama2: Ah, Sirius Black! He's a fascinating character, don't you think? A wizard who's been accused of a crime he didn't commit, and who's been hunted by the authorities for years. He's a complex and conflicted individual, with a troubled past and a deep sense of loyalty to his friends and loved ones.

Sirius Black is a pure-blood wizard, born into an ancient and respected wizarding family. However, his life took a tragic turn when his mother died in a duel with a dark wizard, and his father was killed by the same wizard a few years later. Sirius was left to be raised by his cruel and abusive uncle, who treated him poorly and made him feel like an outcast.

Despite these challenges, Sirius proved to be a gifted student at Hogwarts School of Witchcraft and Wizardry, where he became fast friends with Harry Potter, Ron Weasley, and Hermione Granger. However, his life took a dramatic turn when he was falsely accused of betraying his parents to the dark wizard, Lord Voldemort. He was 

In [43]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=['data/external_response.txt']
).load_data()

index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

In [88]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

response = query_engine.query(query)
print(response)

I'm sorry, but I cannot provide information on "Voldemort" as there is no mention or context provided for this topic in the information given.
